In [1]:
import pandas as pd
import numpy as np
train_set = pd.read_csv('./datasets/final_train_set.csv')
test_set = pd.read_csv('./datasets/final_test_set.csv')

## Logistic Regression

In [2]:
X_train = train_set.drop(['guncelfiyat','Ilan_ID'], axis = 1)
Y_train = train_set['guncelfiyat']

X_test = test_set.drop(['guncelfiyat','Ilan_ID'], axis = 1)
Y_test = test_set['guncelfiyat']

In [ ]:
# ÜZDÜ
# from sklearn.linear_model import LogisticRegression
# logistic_model = LogisticRegression(solver = 'liblinear').fit(X_train,Y_train)

## XGBoost

In [5]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'd:\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [54]:
#Validation function
n_folds = 5
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import xgboost as xgb

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X_train)
    rmse= np.sqrt(-cross_val_score(model, X_train, Y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

In [55]:
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213,
                             random_state =7, nthread = -1)

In [95]:
model_xgb.fit(X_train, Y_train)
xgb_train_pred = model_xgb.predict(X_train)
xgb_pred = model_xgb.predict(X_test)
test_set['guncelfiyat'] = xgb_pred

In [103]:
test_set['guncelfiyat'] = test_set['guncelfiyat'].astype(int)
df = test_set[['guncelfiyat','Ilan_ID']]
df

,guncelfiyat,Ilan_ID
0,656623,64650
1,311999,64651
2,1276837,64652
3,311999,64653
4,86302,64654
...,...,...
35122,1350698,99666
35123,261636,99667
35124,206466,99668
35125,2288881,99656


In [125]:
submission = pd.read_csv('./datasets/sample_submission_zingat.csv')
submission = submission.rename(columns = {'Id': 'Ilan_ID'})
submission = submission.merge(df, how='inner', on='Ilan_ID')
submission = submission.drop('Expected', axis = 1)
submission = submission.rename(columns = {'guncelfiyat': 'Expected'})
submission = submission.rename(columns = {'Ilan_ID': 'Id'})
submission = submission[['Id','Expected']]

In [126]:
submission # Sonuç: 2275555.60719

,Id,Expected
0,64650,656623
1,64651,311999
2,64652,1276837
3,64653,311999
4,64654,86302
...,...,...
35122,99666,1350698
35123,99667,261636
35124,99668,206466
35125,99656,2288881


In [127]:
submission.to_csv('./datasets/final_submission_set.csv', index=False)

In [ ]:
print(rmsle(Y_train, xgb_train_pred))